Na začátku nainstalujeme a importujeme všechny moduly

In [1]:
%pip install numpy pandas scipy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, pearsonr
import matplotlib.pyplot as plt
from random import uniform

Nahrajeme data

In [3]:
df = pd.read_pickle("accidents.pkl.gz")

Následně odfiltrujeme nehody způsobené pod vlivem drogam a všechny sloupce s neduležitými daty.

In [4]:
df = df[(df.p11 != 4) & (df.p11 != 5)]

df = df[['p11', 'p13a', 'p13b']]

df

,p11,p13a,p13b
0,2,0,0
1,2,0,0
2,2,0,0
3,9,0,0
4,0,0,0
...,...,...,...
487156,0,0,0
487157,2,0,0
487158,0,0,0
487159,2,0,0


Prvně vypočítáme Pearsonův korelační koeficient pro určení nějaké korelace.

Pro výpočet korelace použijem dva nové datové sloupce.

- `count` pro počet těžkých zranění a smrtí
- `alc` pro odhadovaný objem alkoholu v krvi

Pro `alc` použijeme mapovací funkci mapující hodnoty ze vstupních dat na promile. Funkce mapuje podle hodnot z dokumentace Statistiky nehodovosti.

In [5]:
def alcohol_map(val):
    if val == 0 or val == 2:  # neznámý nebo žádný alkohol
        return 0
    if val == 1:
        return uniform(0, 0.24)
    if val == 3:
        return uniform(0.24, 0.5)
    if val == 6:
        return uniform(0.5, 0.8)
    if val == 7:
        return uniform(0.8, 1.0)
    if val == 8:
        return uniform(1.0, 1.5)
    if val == 9:
        return uniform(1.5, 2.0)

Provedeme vytoření nových sloupců

In [6]:
df['alc'] = df['p11'].map(alcohol_map)
df['count'] = df['p13a'] + df['p13b']

df

,p11,p13a,p13b,alc,count
0,2,0,0,0.000000,0
1,2,0,0,0.000000,0
2,2,0,0,0.000000,0
3,9,0,0,1.777723,0
4,0,0,0,0.000000,0
...,...,...,...,...,...
487156,0,0,0,0.000000,0
487157,2,0,0,0.000000,0
487158,0,0,0,0.000000,0
487159,2,0,0,0.000000,0


Vypočteme Pearsonův korelační koeficient

In [7]:
pearsonr(df['alc'], df['count'])

(0.0295815207528641, 1.81164602642346e-94)

Výsledek je docela malý, ale pozitivní korelace tam bude, protože p-hodnota je menší než 0.05.

Následně vytvoříme kontingenční tabulku.

In [8]:
sober = df[df.p11 < 7].shape[0]
sober_acc = df[(df.p11 < 7) & (df['count'] > 0)].shape[0]
drunk = df[df.p11 >= 7].shape[0]
drunk_acc = df[(df.p11 >= 7) & (df['count'] > 0)].shape[0]

con_tab = np.array([[sober, sober_acc], [drunk, drunk_acc]])
con_tab

array([[468305,  10777],
       [ 17378,    886]])

Nakonec provedeme test nezávislosti $X^2$

In [9]:
chi2, p, dof, expected = chi2_contingency(con_tab)

display(f'p-value = {p}')

'p-value = 7.626223218709412e-115'

Druhá hodnota vrácená funkcí `chi2_contingency` je p-hodnota testu. 
Tato hdontota je menší než 0.05, takže hypotézu
### Pokud viník nehody byl pod silným vlivem alkoholu, došlo častěji k těžkým zdravotním následkům
můžeme s 95% jistotou potvrdit. 